
## Mini PJT.

- `titanic` competition을 도전해봅시다!

- 이번 프로젝트에서는 간단한 분류 문제를 풀어봅니다.

- sklearn으로 머신러닝 모델을 구현해봅니다.

- Machine Learning Workflow를 따라가봅니다.


Source : https://www.kaggle.com/c/titanic

In [18]:
import os
import pandas as pd

In [19]:
# titanic data 불러오기
base_path = "/Volumes/GoogleDrive/My Drive/fastcamp/datas/titanic/"
os.getcwd()
os.chdir(base_path)

In [21]:
train = pd.read_csv(base_path + "train.csv")
test = pd.read_csv(base_path + "test.csv")
submission = pd.read_csv(base_path + "gender_submission.csv")

In [22]:
# titanic data에서 missing value를 찾아봅니다.
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


### Data Preprocessing

1. 결측치 처리


2. feature selection (분석에 사용하지 않을 column 제거)

In [23]:
# titanic data에서 missing value를 찾아봅니다. 
train.isnull().any(axis=1)

0       True
1      False
2       True
3      False
4       True
       ...  
886     True
887    False
888     True
889    False
890     True
Length: 891, dtype: bool

In [24]:
# Embarked column이 NaN인 row를 찾는다.
# train[train.Embarked.isnull()]

# 최빈값으로 채워보자.
# train.Embarked.value_counts() #"S가 가장 많다."
# train.loc[(train.Pclass == 1) & (train.Sex == "female"), "Embarked"].value_counts() #"S가 가장 많다."

train.loc[train.Embarked.isnull(), "Embarked"] = "S"
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [25]:
# missing value를 handling 해야한다.
# column을 지울까요 / 채울까요?

# drop할 열 골라보자 : 필요없는 정보라고 생각되는 것을 기준으로 선택
# PassengerId, Name, Ticket, Cabin
train =  train.drop(columns = ["PassengerId", "Name", "Ticket", "Cabin"])
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [26]:
# "Age" column 채우기
train = train.fillna(train.Age.mean())
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


### Feature Engineering

1. Categorical feature encoding

2. Normalization

In [27]:
# categorical feature --> One-hot Encoding, Ordinal Encoding

# 1. Ordinal Encoding -> Oridinal feature를 변환할 때 쓰임 : 학력, 선호도, ...
# 2. One-hot Encoding -> Nominal feature를 변환할 때 쓰임 : 성별, 부서, 출신학교 ...

train_OHE = pd.get_dummies(train, columns = ["Sex", "Embarked"])
train_OHE

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.000000,1,0,7.2500,0,1,0,0,1
1,1,1,38.000000,1,0,71.2833,1,0,1,0,0
2,1,3,26.000000,0,0,7.9250,1,0,0,0,1
3,1,1,35.000000,1,0,53.1000,1,0,0,0,1
4,0,3,35.000000,0,0,8.0500,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,27.000000,0,0,13.0000,0,1,0,0,1
887,1,1,19.000000,0,0,30.0000,1,0,0,0,1
888,0,3,29.699118,1,2,23.4500,1,0,0,0,1
889,1,1,26.000000,0,0,30.0000,0,1,1,0,0


In [32]:
# Normalization --> Min-Max scaling를 사용
#!pip install sklearn
from sklearn.preprocessing import MinMaxScaler

# "Survived"는 카테고리 인풋이라 제외한다.
X = train_OHE.drop(columns = "Survived") # input matrix
y = train_OHE.Survived                  # target vector
scaler = MinMaxScaler()
# scaler.fit()
# scaler.transform()
temp = scaler.fit_transform(X.loc[:, ["Age", "Fare"]])
X["Age"] = temp[:, 0]
X["Age"] = temp[:, 1]
X

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,0.014151,1,0,7.2500,0,1,0,0,1
1,1,0.139136,1,0,71.2833,1,0,1,0,0
2,3,0.015469,0,0,7.9250,1,0,0,0,1
3,1,0.103644,1,0,53.1000,1,0,0,0,1
4,3,0.015713,0,0,8.0500,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
886,2,0.025374,0,0,13.0000,0,1,0,0,1
887,1,0.058556,0,0,30.0000,1,0,0,0,1
888,3,0.045771,1,2,23.4500,1,0,0,0,1
889,1,0.058556,0,0,30.0000,0,1,1,0,0


### Training 

In [33]:
# sklearn에서 배웠던 분류 모델들을 불러와봅니다.
from sklearn.linear_model import SGDClassifier # 1. linear classifier
from sklearn.linear_model import LogisticRegression # 2. logistic regression
from sklearn.tree import DecisionTreeClassifier # 3. decision tree
from sklearn.ensemble import RandomForestClassifier # 4. random forest

# 평가 지표
from sklearn.metrics import accuracy_score

In [35]:
clf = SGDClassifier()
clf2 = LogisticRegression()
clf3 = DecisionTreeClassifier()
clf4 = RandomForestClassifier()

clf.fit(X, y)
clf2.fit(X, y)
clf3.fit(X, y)
clf4.fit(X, y)

pred = clf.predict(X)
pred2 = clf2.predict(X)
pred3 = clf3.predict(X)
pred4 = clf4.predict(X)

1. linear classifier, accuracy for training : 0.6162
2. logistic regression, accuracy for training : 0.7980
3. decision tree, accuracy for training : 0.9248
4. random forest, accuracy for training : 0.9248


/usr/local/Caskroom/miniforge/base/envs/fcamp/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [36]:
print("1. linear classifier, accuracy for training : %.4f" % accuracy_score(y, pred))
print("2. logistic regression, accuracy for training : %.4f" % accuracy_score(y, pred2))
print("3. decision tree, accuracy for training : %.4f" % accuracy_score(y, pred3))
print("4. random forest, accuracy for training : %.4f" % accuracy_score(y, pred4))

1. linear classifier, accuracy for training : 0.6162
2. logistic regression, accuracy for training : 0.7980
3. decision tree, accuracy for training : 0.9248
4. random forest, accuracy for training : 0.9248


### Test (Predict)

In [37]:
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [38]:
# test data에 같은 feature engineering을 적용해줍니다.
test = test.drop(columns = ["PassengerId", "Name", "Ticket", "Cabin"])
#test.info()
test = test.fillna(train.Age.mean())
test = test.fillna(train.Fare.mean())

# categorical feature encoding
test_OHE = pd.get_dummies(data = test, columns = ["Sex", "Embarked"])

# normalization
temp = scaler.transform(test_OHE.loc[:, ["Age", "Fare"]])
test_OHE.Age = temp[:, 0]
test_OHE.Fare = temp[:, 1]

In [39]:
# 결과 파일인 submission.csv를 생성합니다.
submission.to_csv(base_path + "submission.csv", index = False)

- 모든 학습이 끝나면 결과를 가지고 제출해볼 수 있습니다.

- 만든 모델 중에 가장 test 성능이 좋은 하나를 제출해볼까요?

[제출하러가기] https://www.kaggle.com/c/titanic